In [99]:
# IMPORT
import os
import time
import sys

In [100]:
# FUNCOES
def Maper(file_comp):
    saida = []
    file = file_comp[1].strip().split('\n')
    filename = file_comp[0]
    filename = os.path.basename(filename)
    filename = filename.replace(".log","")
    for line in file:
        if(len(line) > 0):
            words = line.split()
            if(words[0].isdigit()):
                entrada = filename + '_' + words[0], words[1]
                saida.append(entrada)
    return saida

def Maper_struc(file_comp):
    saida = []
    file = file_comp[1].strip().split('ENDMDL')
    filename = file_comp[0]
    filename = os.path.basename(filename)
    filename = filename.replace(".pdbqt","")
    for line in file:
        if(len(line) > 0):
            words = line.split()
            entrada = filename + '_' + words[1] , line + 'ENDMDL'
            saida.append(entrada)
            #saida.append(filename + '_' + words[1],line)
            #saida.append(line)

    return saida

def reducer(file_comp,valor):
    ID, affinity = file_comp[0],file_comp[1]
    affinity = float(affinity)
    if(affinity <= valor):
        return True
    return False

def criar_mod_inter(line,path_m):
    filename = str(line[0])
    model    = line[1]
    #filename, model = line[0],line[1]
    arq = open(str(path_m + filename + '.pdbqt'), "w")
    arq.writelines(model)
    arq.close()

def criar_mod_inter2(total,path_m):
    for line in total:
        filename = str(line[0])
        model    = line[1]
        arq = open(str(path_m + filename + '.pdbqt'), "w")
        arq.writelines(model)
        arq.close()
    
def saida(lista,path_result):
    arq = open(path_result, 'w')
    for line in lista:
        arq.writelines(line)
    arq.close()
    
def rm(path_local):
    list_arqs = os.listdir(os.path.expanduser(path_local))
    for linha in list_arqs:
        os.remove(path_local + linha)
    
def result(_inicio,_parcial,_fim,_path_mod,_path_result_tempo,_v_affinity,_num):
    tempos = []
    tempos.append('***********************')
    tempos.append('******** SPARK ********')
    tempos.append('***********************')
    
    tempos.append('\n\nMAP E REDUCE\nTEMPO:\t' + str(_parcial - _inicio) )
    tempos.append('\n\nCRIANDO ARQUIVO MODE LOCAL\nTEMPO:\t'  + str(_fim - _parcial) )
    tempos.append('\n\nTEMPO TOTAL\nTEMPO:\t'  + str(_fim - _inicio) )
    tempos.append('\n\nAffinity: ' + str(_v_affinity))
    tempos.append('\n\nQuantidade de Arquivos gravados no diretorio >>')
    tempos.append('\n' + _path_mod)
    tempos.append('\nQtd: ' + str(_num))

    path_result_tempo = _path_result_tempo + 'tempo_result.txt'
    saida(tempos,path_result_tempo)
    

In [101]:
path_local = sys.path[0] + '/'

In [ ]:
os.mkdir( path_local + 'model', 0755 )
os.mkdir( path_local + 'log', 0755 )
os.mkdir( path_local + 'structures', 0755 )
os.mkdir( path_local + 'result', 0755 )

In [103]:
#LISTAR DIRETORIOS
arquivos = os.listdir(os.path.expanduser (path_local))
print arquivos

[u'model', u'log', u'structures', u'result']


In [108]:
#LISTAR ARQUIVOS
arquivos = os.listdir(os.path.expanduser (path_local + 'model'))
print arquivos

[u'ns3pro_dm_0_-_NuBBE_116_obabel_3D_10.pdbqt', u'ns3pro_dm_0_-_NuBBE_4_obabel_3D_16.pdbqt', u'ns3pro_dm_0_-_NuBBE_295_obabel_3D_10.pdbqt', u'ns3pro_dm_0_-_NuBBE_20_obabel_3D_9.pdbqt', u'ns3pro_dm_0_-_NuBBE_20_obabel_3D_1.pdbqt', u'ns3pro_dm_0_-_NuBBE_631_obabel_3D_12.pdbqt', u'ns3pro_dm_0_-_NuBBE_20_obabel_3D_5.pdbqt', u'ns3pro_dm_0_-_NuBBE_348_obabel_3D_1.pdbqt', u'ns3pro_dm_0_-_NuBBE_741_obabel_3D_8.pdbqt', u'ns3pro_dm_0_-_NuBBE_97_obabel_3D_3.pdbqt', u'ns3pro_dm_0_-_NuBBE_348_obabel_3D_5.pdbqt', u'ns3pro_dm_0_-_NuBBE_585_obabel_3D_6.pdbqt', u'ns3pro_dm_0_-_NuBBE_654_obabel_3D_3.pdbqt', u'ns3pro_dm_0_-_NuBBE_594_obabel_3D_12.pdbqt', u'ns3pro_dm_0_-_NuBBE_572_obabel_3D_8.pdbqt', u'ns3pro_dm_0_-_NuBBE_552_obabel_3D_10.pdbqt', u'ns3pro_dm_0_-_NuBBE_461_obabel_3D_9.pdbqt', u'ns3pro_dm_0_-_NuBBE_380_obabel_3D_1.pdbqt', u'ns3pro_dm_0_-_NuBBE_608_obabel_3D_4.pdbqt', u'ns3pro_dm_0_-_NuBBE_550_obabel_3D_16.pdbqt', u'ns3pro_dm_0_-_NuBBE_253_obabel_3D_16.pdbqt', u'ns3pro_dm_0_-_NuBBE_253_obabe

In [109]:
# VARIAVEIS
path_log   = 'swift://log80.spark/'
path_struc = 'swift://structures80.spark/'
path_mod   = path_local + 'model/'
path_result_tempo = path_local + 'result/' 
v_affinity = -3.0

In [106]:
rm(path_mod)

#-------------------------------------------------
inicio = time.time()  # inicio do calculo do tempo
#-------------------------------------------------

log   = sc.wholeTextFiles(path_log,2).flatMap(lambda line1:Maper(line1)).filter(lambda line1:reducer(line1,v_affinity))
struc = sc.wholeTextFiles(path_struc,2).flatMap(lambda line:Maper_struc(line))

model = struc.join(log)

#----------------------------------------------------
parcial = time.time() # tempo parcial - map reduce
#----------------------------------------------------

criar_mod_inter2(model.collect(),path_mod)

#------------------------------------------
fim = time.time() # fim do calculo do tempo
#------------------------------------------

#-------------------------------------------------------------------------------------------------------
num = len(os.listdir(os.path.expanduser(path_mod))) # quantidade de arquivos gravados no diretorio model
#-------------------------------------------------------------------------------------------------------

result(inicio,parcial,fim,path_mod,path_result_tempo,v_affinity,num)

In [110]:
#PRINT
filename = path_local + 'result/tempo_result.txt'
teste = open(filename, "r")
teste = teste.read()
print teste

******************************* SPARK *******************************

MAP E REDUCE
TEMPO:	226.046908855

CRIANDO ARQUIVO MODE LOCAL
TEMPO:	56.3405771255

TEMPO TOTAL
TEMPO:	282.387485981

Affinity: -3.0

Quantidade de Arquivos gravados no diretorio >>
/home/sa470-edf9f3f6d5d4af-05e8d22cb8d/spark-executor-<driver>/userFiles-8e8d7a1d-5eb8-4be2-a2b5-2514fef21f28/model/
Qtd: 8951
